In [1]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os
import glob
import pandas as pd
# ログファイルのパス
log_path = '../../../log/train/**/events.out.tfevents.*'

# マッチするファイルを取得
event_files = glob.glob(log_path, recursive=True)

# 取得したファイルリストを確認
print(f"Found {len(event_files)} event files:")
print(event_files)

# 各イベントファイルを読み取る
df = pd.DataFrame()
for file_path in event_files:
    print(f"Processing file: {file_path}")
    try:
        event_acc = EventAccumulator(file_path)
        event_acc.Reload()  # データをロード
        scalar_tags = event_acc.Tags()['scalars']
        print(f"Scalar tags in {file_path}: {scalar_tags}")
        model_fold = "-".join(file_path.split('/')[2:4])
        for tag in scalar_tags:
            if tag == 'val_corr':
                events = event_acc.Scalars(tag)
                for e in events:
                    df.loc[e.step, model_fold] = e.value
                    print(f"{tag} Step: {e.step}, Value: {e.value}")
    except Exception as e:
        print(f"Error: {e}")
        continue

Found 4 event files:
['../../../log/train/ConvNeXt/fold_2/events.out.tfevents.1733993443.yuhi-aoki-SYS-531A-I.165628.1', '../../../log/train/ConvNeXt/fold_1/events.out.tfevents.1733993253.yuhi-aoki-SYS-531A-I.165628.0', '../../../log/train/ConvNeXt/fold_3/events.out.tfevents.1733993890.yuhi-aoki-SYS-531A-I.165628.2', '../../../log/train/ConvNeXt/fold_4/events.out.tfevents.1733996206.yuhi-aoki-SYS-531A-I.165628.3']
Processing file: ../../../log/train/ConvNeXt/fold_2/events.out.tfevents.1733993443.yuhi-aoki-SYS-531A-I.165628.1
Scalar tags in ../../../log/train/ConvNeXt/fold_2/events.out.tfevents.1733993443.yuhi-aoki-SYS-531A-I.165628.1: ['train_loss', 'val_loss', 'val_corr']
val_corr Step: 0, Value: 0.17795130610466003
val_corr Step: 1, Value: 0.17795130610466003
val_corr Step: 2, Value: 0.17795130610466003
val_corr Step: 3, Value: 0.17795130610466003
val_corr Step: 4, Value: 0.17795130610466003
val_corr Step: 5, Value: 0.934626042842865
val_corr Step: 6, Value: 0.934626042842865
val_cor

In [2]:
df.columns = pd.MultiIndex.from_tuples(df.columns.to_series().str.split("-"), names=['model', 'fold'])


In [3]:
df.sort_index(axis=1, inplace=True)

In [7]:
df

model,..
fold,log
0,-0.204299
1,-0.204299
2,0.774196
3,0.774196
4,0.774196
5,0.965506
6,0.965506
7,0.965506
8,0.965506


In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
# fontlist
from matplotlib.font_manager import FontProperties

# font = FontProperties(fname='/mnt/c/Windows/Fonts/BIZ-UDGothicR.ttc')
# plt.rcParams['font.family'] =  [font.get_name()]
# fm.fontManager.addfont('/mnt/c/Windows/Fonts/BIZ-UDGothicR.ttc')
# 最後の行を取得してインデックスをリセット
last_row = pd.DataFrame(df.iloc[100:169, :].mean().drop(index=["2024120206"]))
mean_values = last_row.groupby('model').mean().round(3).reset_index()


# Seaborn の棒グラフ
plt.figure(figsize=(12, 8))
sns.barplot(data=last_row, x='model', y=0, hue='model')
sns.stripplot(data=last_row, x='model', y=0, dodge=False, size=5, marker='o', linewidth=0, color='black')

# plt.title('Validation Correlation', fontsize=24)
plt.xticks([])
plt.xlabel('')
plt.yticks(fontsize=16)
plt.ylabel("Correlation", fontsize=20)
# テーブルを追加
table_data = mean_values.values
table = plt.table(
    cellText=table_data.T, 
    rowLabels=['Model', 'Average\nCorrelation'], 
    cellLoc='center', 
    loc='bottom',
    bbox=[0, -0.35, 1, 0.3],  # [x, y, width, height]
)

  

table.auto_set_font_size(False)
table.set_fontsize(20)

# グラフとテーブルの間に余白を追加
plt.subplots_adjust(bottom=0.3)
plt.show()

KeyError: '2024120206'